In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def search_amazon(product):
    url = f"https://www.amazon.in/s?k={product}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print("Failed to retrieve data")
        return None

def scrape_product_details(html):
    soup = BeautifulSoup(html, "html.parser")
    products = []
    for item in soup.find_all("div", class_="s-asin"):
        product_details = {}
        product_details['Brand Name'] = item.find("span", class_="a-size-base-plus").text.strip()
        product_details['Name of the Product'] = item.find("span", class_="a-size-medium").text.strip()
        product_details['Price'] = item.find("span", class_="a-offscreen").text.strip()
        product_details['Return/Exchange'] = item.find("span", class_="a-size-small a-color-secondary").text.strip()
        product_details['Expected Delivery'] = item.find("span", class_="a-text-bold").text.strip()
        product_details['Availability'] = item.find("span", class_="a-size-base a-color-success").text.strip()
        product_details['Product URL'] = "https://www.amazon.in" + item.find("a", class_="a-link-normal").get('href')
        products.append(product_details)
    return products

def scrape_first_three_pages(product):
    all_products = []
    for page_num in range(1, 4):
        print(f"Scraping page {page_num}")
        html = search_amazon(product + "&page=" + str(page_num))
        if html:
            products = scrape_product_details(html)
            all_products.extend(products)
    return all_products

def save_to_csv(products, filename):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

def main():
    product = input("Enter the product to search on Amazon: ")
    all_products = scrape_first_three_pages(product)
    save_to_csv(all_products, "amazon_products.csv")

if __name__ == "__main__":
    main()
